# [CS303]Practice 6 - BTS & Practice 7 - Improving BTS
In Practice 6 and 7, we will apply the BTS algorithm and the Improving BTS to solve the N-Queens problem.

In the following code, only two of the function usages are essential. One is `is_valid(self, state)`, which checks if the current state is legal; the other is `is_satisfy(self, state)`, which is to checks if the current board meets the win condition. 
The type of state is [], which stores the tuples(a, b), representing the positions of the queens in it.

In test block, for `Solver`,  `n` indicates the size of the test while `method` indicates which bts function will be used(`bts` or `improving_bts`); for method `solve`, `render` indicates whether to use a graphical interface representation.

## Practice 6 - BTS
You are required to implement complete the function `bts()`.
You can only use iterative way, not recursive. Using recursion will incur a **20% penalty**. You may add any function you want. 
**Remember to test the case where N=6**.

📌 **DDL: Nov.9th**

## Practice 7 - Improving BTS
You are required to complete the function `improving_bts()`.
For `improving_bts()`, you can use one or more of the following strategies: Minimum Remaining Value, Least constraining value, and Forward checking. You should have a good performance (within 2s) **when N = 16 without GUI**. 

📌 **DDL: Nov.16th**

## Evaluation Criteria

Your submission will be assessed on:

* Clear understanding of every line of your code (no blind copy-paste).
* Successful compilation of the program.
* Correctness of the program logic.
* Reasonable efficiency of the solution.

### Grading Policy

* **On-time submission:** 60–100 points
* **Late submission:** 0 points


In [1]:
import numpy as np
import time

def my_range(start, end):
    if start <= end:
        return range(start, end + 1)
    else:
        return range(start, end - 1, -1)


class Problem:
    char_mapping = ('·', 'Q')

    def __init__(self, n=4):
        self.N = n

    def is_valid(self, state):
        """
        check whether the state satisfies the condition or not.
        : param state: list of points (in (row id, col id) tuple form)
        : return: bool value of valid or not
        """
        board = self.get_board(state)
        res = True
        for point in state:
            i, j = point
            condition1 = board[:, j].sum() <= 1
            condition2 = board[i, :].sum() <= 1
            condition3 = self.pos_slant_condition(board, point)
            condition4 = self.neg_slant_condition(board, point)
            res = res and condition1 and condition2 and condition3 and condition4
            if not res:
                break
        return res

    def is_satisfy(self, state):
        return self.is_valid(state) and len(state) == self.N

    def pos_slant_condition(self, board, point):
        i, j = point
        tmp = min(self.N - i - 1, j)
        start = (i + tmp, j - tmp)
        tmp = min(i, self.N - j - 1)
        end = (i - tmp,  j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def neg_slant_condition(self, board, point):
        i, j = point
        tmp = min(i, j)
        start = (i - tmp, j - tmp)
        tmp = min(self.N - i - 1, self.N - j - 1)
        end = (i + tmp, j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def get_board(self, state):
        board = np.zeros([self.N, self.N], dtype=int)
        for point in state:
            board[point] = 1
        return board

    def print_state(self, state):
        board = self.get_board(state)
        print('_' * (2 * self.N + 1))
        for row in board:
            for item in row:
                print(f'|{Problem.char_mapping[item]}', end='')
            print('|')
        print('-' * (2 * self.N + 1))

In [2]:
class Solver:
    def __init__(self, n, method):
        self.N = n
        self.problem = Problem(n)
        self.method = method

    def solve(self, render=True):
        if render:
            import pygame
            w, h = 90 * self.n + 10, 90 * self.N + 10
            screen = pygame.display.set_mode((w, h))
            screen.fill('white')
            action_generator = self.method(self.problem)
            clk = pygame.time.Clock()
            queen_img = pygame.image.load('./queen.png')
            while True:
                for event in pygame.event.get():
                    if event == pygame.QUIT:
                        exit()
                try:
                    actions = next(action_generator)
                    screen.fill('white')
                    for i in range(self.n + 1):
                        pygame.draw.rect(screen, 'black', (i * 90, 0, 10, h))
                        pygame.draw.rect(screen, 'black', (0, i * 90, w, 10))
                    for action in actions:
                        i, j = action
                        screen.blit(queen_img, (10 + 90 * j, 10 + 90 * i))
                    pygame.display.flip()
                except StopIteration:
                    pass
                clk.tick(5)
            pass
        else:
            start_time = time.time()
            for actions in self.method(self.problem):
                pass
            self.problem.print_state(actions)
            print(time.time() - start_time)

# BTS: Backtracking search

In [ ]:
def bts(problem):
    action_stack = []
    N = problem.N
    while not problem.is_satisfy(action_stack):
        # TODO: Implement backtracking search logic here
        if len(action_stack) < N:
            row = len(action_stack)
            for col in range(N):
                action_stack.append((row, col))
                # 合法情况，直接进入下一行
                if problem.is_valid(action_stack):
                    break
                # 不合法，回溯
                action_stack.pop()
            else:
                # 该行没有合法位置，回溯
                if not action_stack:
                    break
                last_action = action_stack.pop()
                while True:
                    row, col = last_action
                    # 尝试下一个位置
                    if col + 1 < N:
                        action_stack.append((row, col + 1))
                        # 合法，直接进入下一行
                        if problem.is_valid(action_stack):
                            break
                        # 回溯
                        last_action = action_stack.pop()
                    else:
                        if not action_stack:
                            break
                        last_action = action_stack.pop()
        else:
            if not action_stack:
                break
            last_action = action_stack.pop()
            while True:
                row, col = last_action
                if col + 1 < N:
                    action_stack.append((row, col + 1))
                    if problem.is_valid(action_stack):
                        break
                    last_action = action_stack.pop()
                else:
                    if not action_stack:
                        break
                    last_action = action_stack.pop()
        yield action_stack
    yield action_stack

# Improving BTS 
* Which variable should be assigned next?
* In what order should its values be tried?
* Can we detect inevitable failure early?

## Minimum Remaining Value
Choose the variable with the fewest legal values in its domain
## Least constraining value
Given a variable, choose the least constraining value: the one that rules out the fewest values in the remaining variables
## Forward checking
Keep track of remaining legal values for the unassigned variables. Terminate when any variable has no legal values.

In [ ]:
def improving_bts(problem):
    action_stack = []
    N = problem.N
    current_col = 0
    while not problem.is_satisfy(action_stack):
        # TODO: Implement improving bts logic here
        if len(action_stack) < N:
            current_row = len(action_stack)
            found_placement = False
            for col in range(current_col, N):
                # 尝试放置皇后
                new_state = action_stack + [(current_row, col)]
                # 合法情况，直接进入下一行，这一行设置为可放置
                if problem.is_valid(new_state):
                    action_stack.append((current_row, col))
                    current_col = 0
                    found_placement = True
                    break
            # 回溯
            if not found_placement:
                if not action_stack:
                    break
                last_queen = action_stack.pop()
                # 继续尝试下一个位置
                current_row, current_col = last_queen[0], last_queen[1] + 1
        else: 
            if not action_stack:
                break
            last_queen = action_stack.pop()
            current_row, current_col = last_queen[0], last_queen[1] + 1
        yield action_stack  
    yield action_stack

## test block

In [5]:
Solver(n=6, method=bts).solve(render=False)

_____________
|·|Q|·|·|·|·|
|·|·|·|Q|·|·|
|·|·|·|·|·|Q|
|Q|·|·|·|·|·|
|·|·|Q|·|·|·|
|·|·|·|·|Q|·|
-------------
0.012430906295776367


In [6]:
Solver(n=16, method=improving_bts).solve(render=False)

_________________________________
|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|
|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|
|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|
|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|
|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|
|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|
|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|
|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|
---------------------------------
12.577753782272339
